# Simple Mean-Reverting Model with and without Transaction Costs

In [61]:
import numpy as np

In [62]:
import pandas as pd

In [63]:
startDate=20060101

In [64]:
endDate=20061231

In [65]:
df=pd.read_table('SPX_20071123.txt')

In [66]:
df['Date']=df['Date'].astype('int')

In [67]:
df.set_index('Date', inplace=True)

In [68]:
df.sort_index(inplace=True)

In [69]:
dailyret=df.pct_change()

In [70]:
marketDailyret=dailyret.mean(axis=1)

In [71]:
# The weights represent the percentage of holdings in your portfolio; negative means short position and positve means long position
# Weights are the difference between dailyret and marketDailyRet
# The negative is added since we want to short positive returns and long negative returns (mean-reverting strategy) 
# Note that marketDailyret is broadcasted to the shape of dailyret
weights=-(np.array(dailyret)-np.array(marketDailyret).reshape((dailyret.shape[0], 1)))

In [72]:
# nansum treats NaN as 0
# Sum of weights for all stocks for each date
wtsum=np.nansum(abs(weights), axis=1)

In [73]:
weights[wtsum==0,]=0

In [74]:
wtsum[wtsum==0]=1 # Avoid dividing by zero error; the results do not change though since 0 / 1 = 0

In [75]:
weights=weights/wtsum.reshape((dailyret.shape[0],1)) # Weighs each stock by daily returns (weights are calculate each trading day)

In [76]:
# Note that shifting the weights is necessary as mentioned in example3_6.ipynb
dailypnl=np.nansum(np.array(pd.DataFrame(weights).shift())*np.array(dailyret), axis=1)

In [77]:
# Only get daily pnl between specified start and end dates
dailypnl=dailypnl[np.logical_and(df.index >= startDate, df.index <= endDate)]

In [78]:
sharpeRatio=np.sqrt(252)*np.mean(dailypnl)/np.std(dailypnl)

In [79]:
sharpeRatio

0.9577856810103862

# With transaction costs

In [80]:
onewaytcost=0.0005

In [81]:
weights=weights[np.logical_and(df.index >= startDate, df.index <= endDate)]

In [82]:
# The weights are shifted because we want to capture the change in the portoflio holdings
# We use abs since we are only concerned about the magnitude of the change (any change incurs transaction cost regardless of direction)
# We are summing along axis=1 since we want to calculate daily transaction cost
dailypnlminustcost=dailypnl - (np.nansum(abs(weights-np.array(pd.DataFrame(weights).shift())), axis=1)*onewaytcost)

In [83]:
sharpeRatioMinusTcost=np.sqrt(252)*np.mean(dailypnlminustcost)/np.std(dailypnlminustcost)

In [84]:
sharpeRatioMinusTcost

-2.161743371896227